In [1]:
cd ..

/gpfs/home/gdb20/code/batch-manager


In [2]:
import input_files
import input_combi
import helpers
import os

In [6]:
global_configs = { '' : {
        'num_cores' : 8,
        'mem_per_cpu_GB' : 4,
        'runtime' : '2-00:00:00',
        #'!max_jobs' : 10, #this doesn't do anything yet
        #'solvent' : 'thf', #this is what Mike told me we use
    }
}

xyz_root = '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules'
mike_dir = os.path.join(xyz_root,'')

mike_mols = input_combi.xyz_files_from_directory(mike_dir)
mike_mols['!directories'] = True

xtb_cm_states = {
    '0_1' : {
        'charge' : 0,
        'spin_multiplicity' : 1,
    },
    '0_3': {
        'charge' : 0,
        'spin_multiplicity' : 3,
    },
}

orca_cm_states = {
    '0_1': {
        'charge' : 0,
        'spin_multiplicity' : 3,
        'broken_symmetry' : True,
    },
    '0_3': {
        'charge' : 0,
        'spin_multiplicity' : 3,
    },
}

xtb_crude_method = {
    'crest_gfn2__gfnff_squick' : {
        'program' : 'CREST',
        'functional' : 'gfn2//gfnff',
        'quick' : 'squick',
        'reopt' : True,
        'cluster' : True,
        'noreftopo' :True #might regret this
        },
    'xtb_gfn2_opt_freq' : {
        'program' : 'XTB',
        'functional' : 'gfn2',
        'run_type' : 'ohess', #opt-freq
        '!coords_from' : '../crest_gfn2__gfnff_squick',
        '!xyz_file' : 'crest_best.xyz',
    },
}
orca_crude_method = {
    'r2scan_3c_sp' : {
        'program' : 'ORCA',
        'functional' : 'r2SCAN-3c',
        'integration_grid' : 'DEFGRID2',
        'scf_tolerance' : 'NormalSCF', #fine if we're not doing a gradient or hessian
        '!coords_from' : '../xtb_gfn2_opt_freq',
        '!xyz_file' : 'xtbopt.xyz',
    },     
    #... this will only print broken symmetry densities. that's annoying.
    #I want to also print triplet densities.
    #what we can do is run a triplet singlepoint, print that,
    #then take the orbitals for BS
    'r2scan_3c_triplet_sp_orbitals' : {        
        'program' : 'ORCA',
        'broken_symmetry' : False,
        'spin_multiplicity' : 3,
        'functional' : 'r2SCAN-3c',
        'basis' : None,
        'aux_basis' : None,
        'integration_grid' : 'DEFGRID2',
        'scf_tolerance' : 'TightSCF',
        'blocks' : {       
            'plots'  : [
                'dim1 100',
                'dim2 100',
                'dim3 100',
                'Format Gaussian_Cube',
                'SpinDens("spindensity");'
                'ElDens("electrondensity");',
            ]
        },   
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'crest_best.xyz'
    },
    'r2scan_3c_bs_singlet_sp_orbitals_occs' : { 
        'program' : 'ORCA',
        'broken_symmetry' : True,
        'spin_multiplicity' : 3,
        'functional' : 'r2SCAN-3c',
        'basis' : None,
        'aux_basis' : None,
        'integration_grid' : 'DEFGRID2',
        'scf_tolerance' : 'TightSCF',
        'blocks' : {
            'output' : [
                'print[P_UNO_OccNum] 1'
            ],                
            'plots'  : [
                'dim1 100',
                'dim2 100',
                'dim3 100',
                'Format Gaussian_Cube',
                'SpinDens("spindensity");'
                'ElDens("electrondensity");',
            ]
        },   
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'crest_best.xyz'
    }
}

crest_gfn2_quick = {
    'crest_gfn2_quick' : {
        'program' : 'CREST',
        'functional' : 'gfn2',
        'quick' : 'quick',
        'reopt' : True,
        'cluster' : True,
        'noreftopo' : True, #might regret this
    },
}
orca_b3lyp = {
    'cam-b3lyp_6-31Gdp_opt_freq' : {
        'program' : 'ORCA',
        'functional' : 'CAM-B3LYP',
        'basis' : '6-31G(d,p)',
        'aux_basis' : 'autoaux',
        'run_type' : 'opt freq', 
        'integration_grid' : 'DEFGRID2', #default settings
        'scf_tolerance' : 'TightSCF', #see if they work in ORCA 6

        '!coords_from' : '../crest_gfn2_quick',
        '!xyz_file' : 'crest_best.xyz',
    }, 
    #just for printing orbitals and getting occupations and the like! 
    #also, this is ALWAYS a broken symmetry job, even if the optimization is from the triplet!
    'cam-b3lyp_6-31Gdp_triplet_sp_orbitals' : {        
        'program' : 'ORCA',
        'broken_symmetry' : False,
        'spin_multiplicity' : 3,
        'functional' : 'r2SCAN-3c',
        'basis' : None,
        'aux_basis' : None,
        'integration_grid' : 'DEFGRID2',
        'scf_tolerance' : 'TightSCF',
        'blocks' : {       
            'plots'  : [
                'dim1 100',
                'dim2 100',
                'dim3 100',
                'Format Gaussian_Cube',
                'SpinDens("spindensity");'
                'ElDens("electrondensity");',
            ]
        },   
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'crest_best.xyz'
    },
    'cam-b3lyp_6-31Gdp_bs_sp_orbitals_occs' : { 
        'program' : 'ORCA',
        'broken_symmetry' : True,
        'spin_multiplicity' : 3,
        'functional' : 'CAM-B3LYP',
        'basis' : '6-31G(d,p)',
        'aux_basis' : 'autoaux',
        'integration_grid' : 'DEFGRID2',
        'scf_tolerance' : 'TightSCF',
        'blocks' : {
            'output' : [
                'print[P_UNO_OccNum] 1'
            ],                
            'plots'  : [
                'dim1 100',
                'dim2 100',
                'dim3 100',
                'Format Gaussian_Cube',
                'SpinDens("spindensity");'
                'ElDens("electrondensity");',
            ]
        },   
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'crest_best.xyz'
    }
}

In [5]:
croot_dir ='/gpfs/research/alabuginlab/gage/michael/crude/'
b3lyp_dir = '/gpfs/research/alabuginlab/gage/michael/quick/'

input_combi.do_everything(
    [global_configs,xtb_cm_states,mike_mols,xtb_crude_method],
    croot_dir,
)

input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,orca_crude_method],
    croot_dir,
)

input_combi.do_everything(
    [global_configs,xtb_cm_states,mike_mols,crest_gfn2_quick],
    b3lyp_dir,
)

input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,orca_b3lyp],
    b3lyp_dir,
)

Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nome2/crest_gfn2__gfnff_squick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nome2/xtb_gfn2_opt_freq
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_tbucore/crest_gfn2__gfnff_squick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_tbucore/xtb_gfn2_opt_freq
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_noby/crest_gfn2__gfnff_squick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_noby/xtb_gfn2_opt_freq
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nomph/crest_gfn2__gfnff_squick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nomph/xtb_gfn2_opt_freq
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nome/crest_gfn2__gfnff_squick
Job Directory NOT WRITTEN at